In [5]:
import numpy as np
import random
import pandas as pd

In [73]:
class GenAlg:

    def __init__(self, population_size, k_crossover, mutation_prob, rates, sigma):
        self.population_size = population_size
        self.k = k_crossover
        self.mutation_prob = mutation_prob
        self.number_of_stocks = len(rates)

        self.rates = rates.copy()
        self.sigma = sigma.copy()

        self.random_pop()
    


    def random_pop(self):
        self.population = np.random.rand(self.population_size,self.number_of_stocks)
        for  i in range(self.population.shape[0]):
            self.population[i,] = self.normalize(self.population[i,])
    
    def normalize(self,vector):
        sum_of_vector = sum(vector)
        for i in range(len(vector)):
            vector[i] =  vector[i]/sum_of_vector
        return vector
    
    def evaluate(self):
        best_fitness = None
        best = None
        for vector in self.population:
            f = self.fitness(vector)
            if best is None or f > best_fitness:
                best = vector
                best_fitness = f
        return best_fitness,best

    def fitness(self,vector):
        rate = np.sum(vector * self.rates)
        volatility = np.sqrt(np.dot(vector.T,np.dot(self.sigma,vector)))

        return rate/volatility

    def get_lucky_one(self):
        random_num = random.randrange(len(self.population))
        return self.population[random_num,:]
    
    def select_parents(self,lucky_one):
        # if lucky_one exists then remove him
        if lucky_one is not None:
            pop = np.ones(shape=(self.population_size-1,self.number_of_stocks))
            i = 0
            for vec in self.population:
                if not (vec == lucky_one).all():
                    pop[i,:] = vec
                    i+=1
        else:
            pop = self.population.copy()
        
        new_pop = np.array(sorted(pop,key=self.fitness,reverse=True))

        return new_pop[:len(new_pop)//2,:]

    def create_children(self,parents):
        
        children = np.zeros(shape=(self.population_size//2,self.number_of_stocks))
        np.random.shuffle(parents)
        first_p, second_p = map(list,np.split(parents,2))
        for i in range(len(parents)//2):
            parent1 = first_p.pop(0)
            parent2 = second_p.pop(0)

            child1, child2 = self.k_crossover(parent1,parent2)

            child1 = self.mutation(child1)
            child2 = self.mutation(child2)

            child1 = self.normalize(child1)
            child2 = self.normalize(child2)

            children[i,:] = child1
            children[i+len(parents)//2,:] = child2

        return children 
    
    def k_crossover(self,x,y):

        slices = set()
        while len(slices) < self.k-1:
            slice_index = random.choice([i for i in range(1,len(x)) if i not in slices])
            if slice_index not in slices:
                slices.add(slice_index)

        slices = sorted(list(slices))
        slices.append(self.number_of_stocks)

        new_x = np.zeros(self.number_of_stocks)
        new_y = np.zeros(self.number_of_stocks)

        first = 0
        switch = True
        for second in slices:
            if switch:
                new_x[first:second] = x[first:second]
                new_y[first:second] = y[first:second]
            else:
                new_x[first:second] = y[first:second]
                new_y[first:second] = x[first:second]
            first = second
            switch = not switch

        return new_x,new_y

    def mutation(self,child):
        for i in range(len(child)):
            if random.random() < self.mutation_prob:
                child[i] += random.gauss(0.25,0.1)
        return child

    def solve(self,max_generations, goal):
        
        best_fit, best = self.evaluate()
        
        for iteration in range(max_generations):
            print(iteration,'iteration',end=' ')
            if best_fit > goal:
                break

            lucky_one = None
            if len(self.population)%2 == 1:
                lucky_one = self.get_lucky_one()

            parents = self.select_parents(lucky_one)

            children = self.create_children(parents)

            if lucky_one is not None:
                lucky_one = lucky_one.reshape((1,len(lucky_one)))
                self.population = np.concatenate((lucky_one,parents,children),axis=0)
            else:
                self.population = np.concatenate((parents,children),axis=0)

            best_fit, best = self.evaluate()
            print('best ratio:',best_fit)
        print('finnished')
        return best,best_fit

In [76]:
wanted_stocks = ['GOOG', 'SPG', 'GOOGL', 'MSFT', 'GD', 'ACN', 'COP', 'F', 'BAC', 'GS',
                'NVDA', 'AIG', 'MS', 'WFC', 'ORCL', 'XOM', 'TGT', 'LOW', 'EXC', 'COST',
                'AXP', 'BK', 'JPM', 'COF', 'CSCO', 'DHR', 'UNH', 'CVS', 'LLY', 'CVX',
                'MET', 'AMT', 'CRM', 'BLK', 'RTX', 'MCD', 'TMO', 'LIN', 'ADBE', 'EMR',
                'USB', 'UPS', 'TSLA', 'PFE', 'PM']
stocks = pd.read_csv('data/sap100_data_08112021.csv',index_col=0).loc[:,wanted_stocks]
returns = stocks/stocks.shift(1)-1
rates = returns.mean() * 252
sigma = returns.cov() * 252
g = GenAlg(population_size=100, k_crossover=5, mutation_prob=0.01, rates=rates,sigma=sigma)

best,best_fit = g.solve(10,8)
print(best,best_fit)
# print(g.population[0,:],'\n',g.population[1,:])

# g.k_crossover(g.population[0,:],g.population[1,:])

# test_pop = np.zeros(shape=(5,45))
# for i in range(5):
#     test_pop[i,i] = 1
# print(test_pop)
# for i in range(5):
#     print(g.fitness(test_pop[i,:]))
# g.population = test_pop
# v = g.get_lucky_one()
# print('lucky one: ',v)
# print('paretns')
# print(g.select_parents(v))

0 iteration best ratio: 3.970766100108268
1 iteration best ratio: 4.035070654199677
2 iteration best ratio: 4.035070654199677
3 iteration best ratio: 4.035070654199677
4 iteration best ratio: 4.035070654199677
5 iteration best ratio: 4.172105168772014
6 iteration best ratio: 4.172105168772014
7 iteration best ratio: 4.220619096166176
8 iteration best ratio: 4.236353419035512
9 iteration best ratio: 4.236353419035512
finnished
[0.0222503  0.00618242 0.14101715 0.02416286 0.0116305  0.00506194
 0.01631985 0.10946576 0.01015295 0.02883629 0.04509702 0.00778493
 0.01378228 0.00226218 0.1263806  0.01668174 0.02242732 0.01481387
 0.01794995 0.02539556 0.03376344 0.02541969 0.00233631 0.0148258
 0.00453587 0.02219779 0.01576228 0.02310145 0.02741787 0.00115135
 0.00111051 0.02453937 0.00353399 0.00867403 0.00756338 0.01400409
 0.024363   0.01353728 0.01197047 0.00415943 0.01186025 0.01791658
 0.00983458 0.00607907 0.00268668] 4.236353419035512


In [47]:
a = [0.01114155, 0.02112081, 0.0026389 , 0.04158689 ,0.02911615 ,0.00810653,
 0.00953936, 0.01291289, 0.03460458, 0.0172124,  0.02812051, 0.00615008,
 0.03882499, 0.01603352 ,0.03222546, 0.02002055, 0.01096212, 0.00879385,
 0.02504149, 0.02297195, 0.04096784, 0.04078883, 0.00404298 ,0.00336519,
 0.02037814, 0.0395612 , 0.041175,   0.03722034, 0.01283837, 0.02809674,
 0.0232359 , 0.03603028 ,0.04134022, 0.00582411, 0.01888116, 0.03858949,
 0.00809903, 0.03437509, 0.03463708 ,0.00438613 ,0.02915961, 0.01188739,
 0.00890677, 0.02284441, 0.01624411]
max(a)

0.04158689

In [65]:
b = [0.00022345, 0.04564275, 0.04497704, 0.04376311, 0.0033903 , 0.03373989,
 0.04167013,0.04474843, 0.03991494, 0.013077 ,  0.04262159 ,0.01209769,
 0.03196004, 0.01024975, 0.03971374 ,0.01291337, 0.00605673, 0.03494775,
 0.01216985, 0.00211064, 0.00081961, 0.03155706, 0.01703879, 0.02700199,
 0.00440683 ,0.01209849, 0.02187265, 0.03169752, 0.02678447 ,0.00618878,
 0.0080071 , 0.03951205 ,0.01762882, 0.01950138, 0.00031214, 0.02950566,
 0.04465236, 0.01634321 ,0.01039963, 0.00432588, 0.02807762, 0.00894588,
 0.03424738, 0.0192949,  0.02379159]
max(b)

0.04564275

In [70]:
def print_stats(vec):
    w = np.array(vec)
    r = np.sum(w*rates)
    print('rate',r)
    v = np.sqrt(np.dot(w.T,np.dot(sigma,w)))
    print('volatility:',v)
    print('shapre ratio',r/v)

In [72]:
print_stats(a)
print()
print_stats(b)

rate 0.4701581146929166
volatility: 0.12546335975815912
shapre ratio 3.747373859580875

rate 0.542997231753761
volatility: 0.1344691863204556
shapre ratio 4.038079255270691
